# Add message history (memory)

The `RunnableWithMessageHistory` let's us add message history to certain types of chains.

Specifically, it can be used for any Runnable that takes as input one of
* a sequence of `BaseMessage`
* a dict with a key that takes a sequence of `BaseMessage`
* a dict with a key that takes the latest message(s) as a string or sequence of `BaseMessage`, and a separate key that takes historical messages

And returns as output one of
* a string that can be treated as the contents of an `AIMessage`
* a sequence of `BaseMessage`
* a dict with a key that contains a sequence of `BaseMessage`

Let's take a look at some examples to see how it works.

## Setup

We'll use Redis to store our chat message histories and Anthropic's claude-2 model so we'll need to install the following dependencies:

In [ ]:
!pip install -U langchain redis anthropic

Set your [Anthropic API  key](https://console.anthropic.com/):

In [ ]:
import getpass
import os

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass()

Start a local Redis Stack server if we don't have an existing Redis deployment to connect to:
```bash
docker run -d -p 6379:6379 -p 8001:8001 redis/redis-stack:latest
```

In [4]:
REDIS_URL = "redis://localhost:6379/0"

### [LangSmith](/docs/langsmith)

LangSmith is especially useful for something like message history injection, where it can be hard to otherwise understand what the inputs are to various parts of the chain.

Note that LangSmith is not needed, but it is helpful.
If you do want to use LangSmith, after you sign up at the link above, make sure to uncoment the below and set your environment variables to start logging traces:

In [2]:
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"]=getpass.getpass()

## Example: Dict input, message output

Let's create a simple chain that takes a dict as input and returns a BaseMessage.

In this case the `"question"` key in the input represents our input message, and the `"history"` key is where our historical messages will be injected.

In [5]:
from typing import Optional

from langchain.chat_models import ChatAnthropic
from langchain.memory.chat_message_histories import RedisChatMessageHistory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.chat_history import BaseChatMessageHistory
from langchain.schema.runnable.history import RunnableWithMessageHistory

In [6]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You're an assistant who's good at {ability}"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = prompt | ChatAnthropic(model="claude-2")

### Defining get_session_history

We need to define a method that takes a thread_id and an optional user_id and based on them returns a `BaseChatMessageHistory`. Given the same inputs, this method should return equivalent outputs.

In [7]:
def get_session_history(
    thread_id: str, *, user_id: Optional[str] = None
) -> BaseChatMessageHistory:
    session_id = thread_id if user_id is None else f"{user_id}:{thread_id}"
    return RedisChatMessageHistory(session_id, url=REDIS_URL)

### Adding message history

To add message history to our original chain we wrap it in the `RunnableWithMessageHistory` class, along with our get_session_history method.

In this case we'll also want to specify `input_messages_key` (the key to be treated as the latest input message) and `history_messages_key` (the key to add historical messages to).

In [8]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
)

## Invoking with config

Whenever we call our chain with message history, we need to include a config that contains the `thread_id`
```python
config={"configurable": {"thread_id": "<THREAD_ID>"}}
```
and optionally the `user_id`
```python
config={"configurable": {"thread_id": "<THREAD_ID>", "user_id": "<USER_ID>"}}
```

Given the same configuration, our chain should be pulling from the same chat message history.

In [9]:
chain_with_history.invoke(
    {"ability": "math", "question": "What does cosine mean?"},
    config={"configurable": {"thread_id": "foobar"}},
)

AIMessage(content=' Cosine is one of the basic trigonometric functions in mathematics. It is defined as the ratio of the adjacent side to the hypotenuse in a right triangle.\n\nSome key facts about cosine:\n\n- It is usually abbreviated as cos.\n\n- For an angle θ in a right triangle, cosine is defined as:\n\ncos(θ) = adjacent / hypotenuse\n\n- Cosine values range from -1 to 1. \n\n- The cosine graph forms a wave that repeats every 2π radians (a full circle).\n\n- Cosine is one of three basic trig functions, along with sine and tangent.\n\n- It has many applications in mathematics, physics, engineering, and other fields involving triangles, waves, and periodic motions.\n\n- Important uses include finding side lengths in right triangles, modeling periodic phenomena like sound/light waves, calculating AC electrical power, and more.\n\nSo in summary, cosine is a fundamental trigonometric function that relates the angles and side lengths of right triangles. It measures the ratio of the adj

In [10]:
chain_with_history.invoke(
    {"ability": "math", "question": "What's its inverse"},
    config={"configurable": {"thread_id": "foobar"}},
)

AIMessage(content=' The inverse of the cosine function is called the arccosine or inverse cosine, often denoted as cos-1 or arccos. \n\nThe arccosine function takes a value between -1 and 1 as input and returns the angle (in radians) whose cosine is that input value.\n\nSome key facts about arccosine:\n\n- It is the inverse function of cosine, which means: \n\ncos(arccos(x)) = x, for -1 ≤ x ≤ 1\n\nand\n\narccos(cos(θ)) = θ\n\n- The range of the arccosine function is 0 to π radians (0 to 180 degrees).\n\n- It is a many-to-one function, meaning there are multiple angles that can have the same cosine value. \n\n- To find the principal (primary) arcCosine angle between 0 and π:\n\narccos(x) = θ such that cos(θ) = x\n\n- To find other angles with the same cosine: \n\nθ = 2πn ± arccos(x)  for any integer n\n\n- The derivative of arccos(x) is -1/')

:::tip [Langsmith trace](https://smith.langchain.com/public/863a003b-7ca8-4b24-be9e-d63ec13c106e/r)
:::

Looking at the Langsmith trace for the second call, we can see that when constructing the prompt, a "history" variable has been injected which is a list of two messages (our first input and first output).

## Example: messages input, dict output

In [14]:
from langchain.schema.messages import HumanMessage
from langchain.schema.runnable import RunnableMap

chain = RunnableMap({"output_message": ChatAnthropic(model="claude-2")})
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    output_messages_key="output_message",
)

chain_with_history.invoke(
    [HumanMessage(content="What did Simone de Beauvoir believe about free will")],
    config={"configurable": {"thread_id": "baz"}},
)

{'output_message': AIMessage(content=' Here is a summary of Simone de Beauvoir\'s views on free will:\n\n- De Beauvoir was an existentialist philosopher and believed strongly in the concept of free will. She rejected the idea that human nature or instincts determine behavior.\n\n- Instead, de Beauvoir argued that human beings define their own essence or nature through their actions and choices. As she famously wrote, "One is not born, but rather becomes, a woman."\n\n- De Beauvoir believed that while individuals are situated in certain cultural contexts and social conditions, they still have agency and the ability to transcend these situations. Freedom comes from choosing one\'s attitude toward these constraints.\n\n- She emphasized the radical freedom and responsibility of the individual. We are "condemned to be free" because we cannot escape making choices and taking responsibility for our choices. \n\n- De Beauvoir felt that many people evade their freedom and responsibility by adop

In [16]:
chain_with_history.invoke(
    [HumanMessage(content="How did this compare to Sartre")],
    config={"configurable": {"thread_id": "baz"}},
)

{'output_message': AIMessage(content=" There are many similarities between Simone de Beauvoir's views on free will and those of Jean-Paul Sartre, though some key differences emerge as well:\n\nSimilarities with Sartre:\n\n- Both were existentialist thinkers who rejected determinism and emphasized human freedom and responsibility.\n\n- They agreed that existence precedes essence - there is no predefined human nature that determines who we are.\n\n- Individuals must define themselves through their choices and actions. This leads to anxiety but also freedom.\n\n- The human condition is characterized by ambiguity and uncertainty, rather than fixed meanings/values.\n\n- Both felt that most people evade their freedom through self-deception, conformity, or adopting collective identities/values uncritically.\n\nDifferences from Sartre: \n\n- Sartre placed more emphasis on the burden and anguish of radical freedom. De Beauvoir focused more on its positive potential.\n\n- De Beauvoir critiqued S

:::tip [LangSmith trace](https://smith.langchain.com/public/f6c3e1d1-a49d-4955-a9fa-c6519df74fa7/r)
:::

## More examples

We could also do any of the below:

In [ ]:
from operator import itemgetter

# messages in, messages out
RunnableWithMessageHistory(
    ChatModel,
    get_session_history,
)

# dict with single key for all messages in, messages out
RunnableWithMessageHistory(
    itemgetter("input_messages") | ChatModel,
    get_session_history,
    input_messages_key="input_messages",
)